# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

## Basic FABRIC Slice Configuration

In [ ]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [ ]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'P4Lang_Tutorial'

[site1,site2,site3] = fablib.get_random_sites(count=3)
print(f"Sites: {site1},{site2},{site3}")

# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 100

# Hosts
h1_name = "h1"
h2_name = "h2"
h3_name = "h3"

h1_subnet=IPv4Network('10.0.1.0/24')
h1_addr=IPv4Address('10.0.1.1')

h2_subnet=IPv4Network('10.0.2.0/24')
h2_addr=IPv4Address('10.0.2.2')

h3_subnet=IPv4Network('10.0.3.0/24')
h3_addr=IPv4Address('10.0.3.3')

host_cores = 2
host_ram = 8
host_disk = 10

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'
net_h3_name = 'net_h3'

net_s1_s2_name = 'net_s1_s2'
net_s2_s3_name = 'net_s2_s3'
net_s1_s3_name = 'net_s1_s3'

# All node properties
image = 'default_ubuntu_20'

### Create the Slice

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site1,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1_iface_local = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()[0]
    s1_iface_to_s2 = s1.add_component(model='NIC_Basic', name="s1_switch_nic1").get_interfaces()[0]
    s1_iface_to_s3 = s1.add_component(model='NIC_Basic', name="s1_switch_nic2").get_interfaces()[0]

    # Add switch node s2
    s2 = slice.add_node(name=s2_name, site=site2,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s2_iface_local = s2.add_component(model='NIC_Basic', name="s2_local_nic").get_interfaces()[0]
    s2_iface_to_s1 = s2.add_component(model='NIC_Basic', name="s2_switch_nic1").get_interfaces()[0]
    s2_iface_to_s3 = s2.add_component(model='NIC_Basic', name="s2_switch_nic2").get_interfaces()[0]
    
    # Add switch node s3
    s3 = slice.add_node(name=s3_name, site=site3,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s3_iface_local = s3.add_component(model='NIC_Basic', name="s3_local_nic").get_interfaces()[0]
    s3_iface_to_s1 = s3.add_component(model='NIC_Basic', name="s3_switch_nic1").get_interfaces()[0]
    s3_iface_to_s2 = s3.add_component(model='NIC_Basic', name="s3_switch_nic2").get_interfaces()[0]    
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h1_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h2_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    
    # Add host node h3
    h3 = slice.add_node(name=h3_name, site=site3, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h3_iface = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()[0]
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[s1_iface_to_s2, s2_iface_to_s1])
    swtich_net2 = slice.add_l2network(name=net_s2_s3_name, interfaces=[s2_iface_to_s3, s3_iface_to_s2])
    swtich_net3 = slice.add_l2network(name=net_s1_s3_name, interfaces=[s3_iface_to_s1, s1_iface_to_s3])

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_local, h1_iface])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s2_iface_local, h2_iface])
    host_net3 = slice.add_l2network(name=net_h3_name, interfaces=[s3_iface_local, h3_iface])
    
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

## Observe the Slice's Attributes

### Print the slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")

### Print the Node List

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

### Print the Node Details

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

### Print the Node SSH Commands

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node.get_name()}: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

### Print the Interfaces

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

## Configure Nodes


In [ ]:
config_threads = {}

In [ ]:
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/" 

try:
    
    
    h1 = slice.get_node(name=h1_name)        
    if type(ip_address(h1.get_management_ip())) is IPv6Address:
        h1.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
       
    h1_os_iface = h1.get_interface(network_name=net_h1_name)
    h1_os_iface.ip_addr_add(addr=h1_addr, subnet=h1_subnet)
    h1_config_thread = h1.execute_thread(host_config_script)
    config_threads[h1] = h1_config_thread

    
    h2 = slice.get_node(name=h2_name)
    if type(ip_address(h2.get_management_ip())) is IPv6Address:
        h2.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h2_os_iface = h2.get_interface(network_name=net_h2_name)
    h2_os_iface.ip_addr_add(addr=h2_addr, subnet=h2_subnet)
    h2_config_thread = h2.execute_thread(host_config_script)
    config_threads[h2] = h2_config_thread

    h3 = slice.get_node(name=h3_name)
    if type(ip_address(h3.get_management_ip())) is IPv6Address:
        h3.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h3_os_iface = h3.get_interface(network_name=net_h3_name)
    h3_os_iface.ip_addr_add(addr=h3_addr, subnet=h3_subnet)
    h3_config_thread = h3.execute_thread(host_config_script)
    config_threads[h3] = h3_config_thread

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



Below are commands to let sudo work with the global proxy.

In [ ]:
try:
    s1 = slice.get_node(name=s1_name)
    s1_h1_os_iface = s1.get_interface(network_name=net_h1_name)
    s1_s2_os_iface = s1.get_interface(network_name=net_s1_s2_name)
    s1_s3_os_iface = s1.get_interface(network_name=net_s1_s3_name)
        
    if type(ip_address(s1.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
    
    file_attributes = s1.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s1_h1_os_iface.get_os_interface()} {s1_s2_os_iface.get_os_interface()} {s1_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'"
    s1_config_thread = s1.execute_thread(command)
    config_threads[s1] = s1_config_thread


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [ ]:
try:
    s2 = slice.get_node(name=s2_name)
    s2_h2_os_iface = s2.get_interface(network_name=net_h2_name)
    s2_s1_os_iface = s2.get_interface(network_name=net_s1_s2_name)
    s2_s3_os_iface = s2.get_interface(network_name=net_s2_s3_name)

    if type(ip_address(s2.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
    
    file_attributes = s2.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command= f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s2_h2_os_iface.get_os_interface()} {s2_s1_os_iface.get_os_interface()} {s2_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'"
    s2_config_thread = s2.execute_thread(command)
    config_threads[s2] = s2_config_thread

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [ ]:
try:
    s3 = slice.get_node(name=s3_name)
    s3_h3_os_iface = s3.get_interface(network_name=net_h3_name)
    s3_s1_os_iface = s3.get_interface(network_name=net_s1_s3_name)
    s3_s2_os_iface = s3.get_interface(network_name=net_s2_s3_name)

    if type(ip_address(s3.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
     
    file_attributes = s3.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command = f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s3_h3_os_iface.get_os_interface()} {s3_s1_os_iface.get_os_interface()} {s3_s2_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
    s3_config_thread = s3.execute_thread(command)
    config_threads[s3] = s3_config_thread


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [ ]:
try:
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        print(f"stdout: {stdout}")
        print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 


### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [ ]:
for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = slice.get_node(name=switch_name)
    management_ip_switch = str(switch_node.get_management_ip())
    print("Swtitch Name        : {}".format(switch_node.get_name()))
    print("Management IP    : {}".format(management_ip_switch))
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}_commands.txt'
    print(cmd_file)
    file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

## The switches are now configured and running. Now we are going to send packets over the switches.

We are going to use `send.py` and `receive.py`. We are going to re-upload them to the servers and use them. Make sure to modify the interface names in the script accordingly.

In [ ]:
h1.upload_file('scripts/send.py', 'tutorials/exercises/basic_tunnel/send_modified.py')
h2.upload_file('scripts/receive.py', 'tutorials/exercises/basic_tunnel/receive_modified.py')

#h1.execute('sudo mv send.py tutorials/exercises/basic_tunnel/send_modified.py')
#h2.execute('sudo mv receive.py tutorials/exercises/basic_tunnel/receive_modified.py')

## Below, we send a few packets.

In [ ]:
h1.execute('tmux new -d \'timeout 30 watch -n 5 sudo python3 tutorials/exercises/basic_tunnel/send_modified.py --dst_id 2 10.10.2.2 "message100"\'')

## And below we receive them.

In [ ]:
print(h2.execute('sudo timeout 30 sudo python3 tutorials/exercises/basic_tunnel/receive_modified.py')[0])

## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()